In [ ]:
!python3 -m pip install -U git+https://github.com/facebookresearch/demucs#egg=demucs

In [1]:
import torch
import numpy as np
import torchaudio
import tempfile
from cdpam import CDPAM
from demucs.separate import Separator
from IPython.display import Audio
from datasets import load_dataset, Dataset
from walloc import walloc
from spauq.core.metrics import spauq_eval
from fastprogress.fastprogress import progress_bar
class Config: pass

In [2]:
device = "cuda"
separator = Separator()
cdpam_loss = CDPAM()
MUSDB = load_dataset("danjacobellis/musdb18HQ", split='validation')

/home/dgj335/.local/lib/python3.10/site-packages/cdpam/cdpam.py:45: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(modfolder,map_location="cpu")['state']


Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/22 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/22 [00:00<?, ?it/s]

In [3]:
def pad(audio, p=2**16):
    B,C,L = audio.shape
    padding_size = (p - (L % p)) % p
    if padding_size > 0:
        audio = torch.nn.functional.pad(audio, (0, padding_size), mode='constant', value=0)
    return audio

In [4]:
max_duration=44100*180
SSDR = []
SRDR = []
PSNR = []
CDPAM_dB = []
# res_red = torchaudio.transforms.Resample(44100, 16536).to(device)
# res_inc = torchaudio.transforms.Resample(16536, 44100).to(device)
with torch.no_grad():
    for i_sample in progress_bar(range(0, len(MUSDB), 5)):
        SSDR.append([])
        SRDR.append([])
        PSNR.append([])
        CDPAM_dB.append([])
        y = []
        for i_instr in range(5):
            sample = MUSDB[i_sample+i_instr]
            instr = sample['instrument']
            x, fs = torchaudio.load(sample['audio']['bytes'])
            x = x[:,:max_duration]
            L = x.shape[-1]
            x_padded = pad(x.unsqueeze(0), 2**16).to(device)
            if i_instr == 0:
                # mix = res_inc(res_red(x_padded[0]))
                mix = x_padded[0]
                pred = separator.separate_tensor(mix)
            else:
                ℓ = min(x_padded.shape[-1], pred[1][instr].shape[-1])
                SDR = spauq_eval(
                    reference=x_padded[0,:,:ℓ].to("cpu"),
                    estimate=pred[1][instr][:,:ℓ].to("cpu"),
                    fs = fs
                )
                psnr = 20*np.log10(2) - 10*np.log10(torch.nn.functional.mse_loss(
                    x_padded[0,:,:ℓ],
                    pred[1][instr][:,:ℓ]
                ).item())
                cdpam = cdpam_loss.forward(
                    wav_in=x_padded[0,:,:ℓ],
                    wav_out=pred[1][instr][:,:ℓ]
                )
                SSDR[-1].append(SDR['SSR'])
                SRDR[-1].append(SDR['SRR'])
                PSNR[-1].append(psnr)
                CDPAM_dB[-1].append(-np.log10(cdpam.mean().item()))
SSDR = torch.tensor(SSDR)
SRDR = torch.tensor(SRDR)
PSNR = torch.tensor(PSNR)
CDPAM_dB = torch.tensor(CDPAM_dB)

/home/dgj335/.local/lib/python3.10/site-packages/spauq/core/preprocessing.py:325: UserWarning: No forgive_mode specified, defaulting to `none`
  warnings.warn(


In [5]:
metrics = {
    'SSDR': SSDR,
    'SRDR': SRDR,
    'PSNR': PSNR,
    'CDPAM_dB': CDPAM_dB,
}

instruments = ['other', 'drums', 'bass', 'vocals']

data = {}
for metric_name, metric_tensor in metrics.items():
    for i, instr in enumerate(instruments):
        column_name = f"{metric_name}_{instr}"
        data[column_name] = metric_tensor[:, i].tolist()
dataset = Dataset.from_dict(data)

In [6]:
dataset.push_to_hub("danjacobellis/LSDIR_demucs_orig",split='validation')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/danjacobellis/LSDIR_demucs_orig/commit/2c88071cf9f7b3757291d15ffe585de9fece21d0', commit_message='Upload dataset', commit_description='', oid='2c88071cf9f7b3757291d15ffe585de9fece21d0', pr_url=None, pr_revision=None, pr_num=None)

In [7]:
dataset.to_pandas().mean(axis=0)

SSDR_other         13.753074
SSDR_drums         22.302781
SSDR_bass          15.660749
SSDR_vocals        14.246989
SRDR_other          5.007309
SRDR_drums          9.913120
SRDR_bass           0.768342
SRDR_vocals        -0.350839
PSNR_other         37.382034
PSNR_drums         40.244511
PSNR_bass          40.930796
PSNR_vocals        41.371783
CDPAM_dB_other      4.705127
CDPAM_dB_drums      4.786927
CDPAM_dB_bass       4.951358
CDPAM_dB_vocals     4.920378
dtype: float64

In [8]:
dataset.to_pandas()

,SSDR_other,SSDR_drums,SSDR_bass,SSDR_vocals,SRDR_other,SRDR_drums,SRDR_bass,SRDR_vocals,PSNR_other,PSNR_drums,PSNR_bass,PSNR_vocals,CDPAM_dB_other,CDPAM_dB_drums,CDPAM_dB_bass,CDPAM_dB_vocals
0,12.033877,30.183285,22.677050,11.067833,5.116219,13.361151,10.309665,4.878315,38.447182,41.221790,41.491991,41.366988,3.122452,4.871617,5.117370,4.855479
1,15.788874,17.393529,21.988362,20.345934,7.106019,7.036381,10.731623,9.255585,37.012966,39.045458,43.287259,39.997815,4.861599,3.907997,4.728462,3.967142
2,19.049195,14.383686,11.243682,7.746423,7.801902,5.681356,5.219229,0.575471,35.297462,41.342185,38.395870,38.776538,4.796523,4.767289,4.585105,5.128774
3,11.980566,21.492515,12.902461,0.043586,5.565767,9.313979,5.302225,-23.820039,35.940989,37.379273,35.163991,40.720145,4.393557,4.529362,5.066222,5.078483
4,17.163697,10.290719,13.856346,8.171781,6.934570,4.150102,6.720999,3.853833,43.047518,48.072475,43.224790,50.167419,5.554521,5.657174,5.257745,5.708686
5,15.293042,34.719031,27.772527,0.000000,7.034896,16.188921,16.353978,-80.000000,41.371122,44.511579,41.539437,58.727607,4.745759,5.684340,5.080981,6.070775
6,17.129637,25.684939,9.874755,13.278104,6.485002,10.922376,4.249162,5.641306,35.936714,41.999283,43.172275,37.333089,5.138228,4.793599,5.196582,4.564872
7,24.078942,16.341371,22.733632,10.631137,12.123110,8.463392,10.963991,2.791245,38.397314,46.043490,44.274626,39.280651,5.227454,4.972369,4.180304,5.018390
8,8.348634,15.860092,22.095787,19.079625,2.323870,6.336440,11.845893,8.600459,36.158959,40.129608,41.646698,36.770654,4.489866,4.801585,5.125073,4.675937
9,15.247323,22.182182,7.356576,15.793053,6.833374,10.057064,2.046529,7.312038,35.990099,40.573443,39.024451,39.199085,4.572828,4.876168,4.685474,4.986523
